# Agregando os dados

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

## 1 - Familiarizando com os dados

In [ ]:
# Read a space-separated text file
df = pd.read_csv('../data/01_raw/wider/cl_cmb_c1_m1.dat',  sep=r'\s+', header = None)

print(df)

In [ ]:
df.isnull().sum()

In [ ]:
# Assign new headers
df.columns = ['cl_e', 'cl_b']

print("\nDataFrame with headers:")

print(df)

In [ ]:
# Read a space-separated text file
df2 = pd.read_csv('../data/01_raw/wider/cl_cmb_c1_m2.dat',  sep=r'\s+', header = None)

print(df2)

In [ ]:
df2.isnull().sum()

### Observações

- Não há dados nulos nos dois arquivos analisados.

- Observa-se que as duas primeiras linhas são iguais a zero. 

- Ao conversar com a pessoa que gerou os dados através de uma simulação, ficou recomendado que ambas linhas podem ser excluídas do dataset. 

- Checa-se mais adiante se o dataframe final, contendo a união de todos os arquivos disponíveis, não terá dados nulos.

In [ ]:
# Deletando colunas desnecessárias
df2.drop(index=[0, 1], inplace=True)

print(df2)

In [ ]:
# Assign new headers
df2.columns = ['cl_e', 'cl_b']

print("\nDataFrame with headers:")

print(df2)

# 2 - Juntando-se os dados

Foram disponibilizados vários arquivos de simulação para os valores de multipolos do espectro de potência da radiação cósmica de fundo.

Para cada valor de um observável que queremos prever, foram feitas 10 simulações.

Abaixo une-se os 10 arquivos que serão referentes a um único valor de observável cosmológico (que será o target).

Exclui-se as linhas com valores iguais a 0: linhas 0,1 e 512 



In [ ]:
directory = '../data/01_raw/wider/'
dataframes = []
base_columns = ['cl_e', 'cl_b']

for i in range(1, 11):
    file_name = f'cl_cmb_c501_m{i}.dat'
    file_path = os.path.join(directory, file_name)

    try:
        df = pd.read_csv(file_path, sep='\s+', header=None)
        # Assign new headers
        df.columns = [f'cl_e_r{i}', f'cl_b_r{i}']


        df.drop(index=[0, 1, 512], inplace=True)
        
        dataframes.append(df)
        print(f"Prepared {file_name}")
    except FileNotFoundError:
        print(f"Error: The file {file_name} was not found.")
    except Exception as e:
        print(f"An error occurred while reading {file_name}: {e}")

In [ ]:
# Join all DataFrames side-by-side using axis=1
combined_df = pd.concat(dataframes, axis=1)

# Print the shape of the final DataFrame to see the total number of columns
print(f"\nCombined DataFrame has {combined_df.shape[0]} rows and {combined_df.shape[1]} columns.")

# Display the first few rows of the final DataFrame to show the side-by-side join
print(combined_df.head())

# 3 - Transpondo os dados

- Para utilizar todos os dados e simular a presença de ruído nos dados, abaixo transpõe-se os dados lidos:

In [ ]:
# Create the DataFrame for cl_e_avg by transposing the column
df_transposed = combined_df.T.reset_index()

df_transposed

In [ ]:
columns = ['index'] + [f'l_{i}' for i in range(2,512)]


# Assign new headers
df_transposed.columns = columns

print("\nDataFrame with headers:")

print(df_transposed)

# 4 - Separando o dataframe acima em dois

- As 510 colunas do dataframe acima correspondem aos momentos de multipole do modo $E$ e modo $B$ no espectro de potências da CMB.

- Os 510 momentos podem ser utilizados para predizer o valor do observável $r$ no caso dos modos $B$ e para predizer o observável $\tau$ no caso do modo $E$.

- Por isso separa-se o dataframe acima em dois e toma-se a transposta dos dataframes resultantes

In [ ]:
df_e = df_transposed.loc[df_transposed['index'].str.startswith('cl_e'),:]

In [ ]:
df_e

In [ ]:
df_b = df_transposed.loc[df_transposed['index'].str.startswith('cl_b'),:]
df_b

# 5 - Incluindo mais dados

- O processo acima foi para exemplificar o pré-processamento referente a uma linha das features do dataframe que será usado para a modelagem

- Abaixo inclui-se os demais dados seguindo-se a mesma linha de raciocínio

In [ ]:
directory = '../data/01_raw/wider/'

dataframes = []

# Outer loop for 'j' values
for j in range(1, 1001):  # j from 501 to 700


    # Inner loop for 'i' values
    for i in range(1, 11):  # i from 1 to 10
        file_name = f'cl_cmb_c{j}_m{i}.dat'
        file_path = os.path.join(directory, file_name)

        try:
            df = pd.read_csv(file_path, sep='\s+', header=None)
            df.columns = [f'cl_e_realization{i}_target{j}', f'cl_b_realization{i}_target{j}']
            df.drop(index=[0, 1,512], inplace=True)
            dataframes.append(df)
            print(f"Prepared {file_name}")
        except FileNotFoundError:
            print(f"Error: The file {file_name} was not found.")
        except Exception as e:
            print(f"An error occurred while reading {file_name}: {e}")

if dataframes:
    combined_df = pd.concat(dataframes, axis=1)  

else:
    print(f"\nNo dataframes were created for c{j}.")

# Create the DataFrame for cl by transposing the column
df_transposed = combined_df.T.reset_index()

columns = ['index'] + [f'l_{i}' for i in range(2,512)]

# Assign new headers
df_transposed.columns = columns   

# After the loop, separetes the dataframes into b e e modes
df_e = df_transposed.loc[df_transposed['index'].str.startswith('cl_e'),:]
df_b = df_transposed.loc[df_transposed['index'].str.startswith('cl_b'),:]

print("\n" + "="*40)
print("Final Concatenated DataFrame for cl_e_avg:")
print(df_e)

print("\n" + "="*40)
print("Final Concatenated DataFrame for cl_b_avg:")
print(df_b)
        

# 6 - Juntando os targets

## 6.1 - Extendendo o número de linhas dos targets

No dataset da feature, as 10 realizações associadas ao mesmo índice de target estarão associadas as linhas do target abaixo.

Portanto, será necessário extender as linhas dos targets antes de juntá-los ao dataframe das features.

In [ ]:
df = pd.read_csv('../data/01_raw/targets/CosmoID_r_tau_As_1to1000_concat_2dLHsampling_wider.txt', sep=r'\s+', header=None)
df


Adicionando nome às colunas

In [ ]:
# Assign new headers
df.drop(columns=[0], inplace=True)
df.columns = ['r', 'tau', 'As']

print("\nDataFrame with headers:")

print(df)

Extendendo o número de linhas dos targets:

In [ ]:
# n is the number of times you want to repeat each row
n = 10

# Use index.repeat() to duplicate the index n times,
# then use .loc[] to create the new DataFrame.
df_extended = df.loc[df.index.repeat(n)]

# Reset the index for a clean 0 to 99 range
df_extended = df_extended.reset_index(drop=True)

In [ ]:
df_extended

In [ ]:
# Before concatenation, reset the index of each DataFrame
final_e_reset = df_e.reset_index(drop=True)
final_b_reset = df_b.reset_index(drop=True)

df_reset = df_extended.reset_index(drop=True)

# Now concatenate the DataFrames with the new, unique index
final_b = pd.concat([final_b_reset, df_reset], axis=1)
final_b.drop(columns=['tau','As'], inplace=True)


# Now concatenate the DataFrames with the new, unique index
final_e = pd.concat([final_e_reset, df_reset], axis=1)
final_e.drop(columns=['r','As'], inplace=True)


final_b


In [ ]:
final_e

# Separando entre treino e teste

Como o conjunto de dados utilizados acima tem a peculiaridade de ter 10 observações associadas ao mesmo target, abaixo utiza-se a seguinte estratégia:

- Divide-se os datasets entre X para as features e y para os targets.

- Utiliza-se a classe GroupShuffleSplit para encontrar-se índices agrupados utilizando-se os valores únicos do target para o agrupamento do dataframe.

- Filtra-se as features e targets utilizando-se os índices obtidos acima.

- Concatena-se as features com os targets para obter um dataframe final.

- Aplica-se a divisão entre treino e teste utilizando-se as funções criadas para excutar as etapas descritas acima.

In [ ]:
X_features_e = final_e.drop(columns=['index','tau'])
y_target_e = final_e['tau']

X_features_b = final_b.drop(columns=['index','r'])
y_target_b = final_b['r']

# O array 'groups_e' é o valor de 'tau' para cada linha. 
# O array 'groups_b' é o valor de 'r' para cada linha. 
# O GroupKFold garantirá que todos os valores idênticos sejam mantidos juntos.
groups_e = y_target_e.to_numpy().ravel()
groups_b = y_target_b.to_numpy().ravel()


In [ ]:
def gerar_indices_gss(X_features: np.ndarray, y_target: np.ndarray, train_ratio: float = 0.8) -> tuple:
    """
    Gera os índices de treino e teste usando GroupShuffleSplit (80/20 por padrão).
    """
    groups = y_target.to_numpy().ravel()
    
    # GroupShuffleSplit para um único split (n_splits=1) com a proporção train_ratio
    gss_splitter = GroupShuffleSplit(
        n_splits=1, 
        train_size=train_ratio, 
        random_state=42 # Fixa a aleatoriedade da divisão para reprodutibilidade
    )
    
    # GSS.split() retorna um iterador. Pegamos o primeiro (e único) par de índices.
    for train_index, test_index in gss_splitter.split(X_features, groups=groups):
        return train_index, test_index
    
    return np.array([]), np.array([])


def aplicar_split(X: np.ndarray, y: np.ndarray, 
                  train_index: np.ndarray, test_index: np.ndarray) -> tuple:
    """
    Fatia os arrays X e Y nos conjuntos de Treino e Teste usando os índices fornecidos.
    """
    X_train_array = X.iloc[train_index]
    X_test_array = X.iloc[test_index]
    
    y_train_array = y.iloc[train_index]
    y_test_array = y.iloc[test_index]
    
    return X_train_array, X_test_array, y_train_array, y_test_array


def montar_dataframe(X_array: np.ndarray, y_array: np.ndarray) -> pd.DataFrame:
    """
    Monta um DataFrame combinando features (X) e target (Y).
    """
    #df_features = pd.DataFrame(X_array, columns=colunas_X)
    #df_target = pd.DataFrame(y_array, columns=[coluna_y])
    
    return pd.concat([X_array, y_array], axis=1)

def criar_datasets_treino_teste_final(X_features: np.ndarray, 
                                     y_target: np.ndarray):                        
    """
    Função principal que orquestra o split GKF e a montagem dos DataFrames.
    """
    
    # 1. Geração dos Índices 
    train_index, test_index = gerar_indices_gss(X_features, y_target)
    
    # 2. Aplicação do Split 
    X_train, X_test, y_train, y_test = aplicar_split(
        X_features, y_target, train_index, test_index
    )
    
    # 3. Montagem dos DataFrames 
    df_train = montar_dataframe(X_train, y_train)
    df_test = montar_dataframe(X_test, y_test)
    
    return df_train, df_test


In [ ]:
# 1. Processar os Dados do MODO E
df_train_E, df_test_E = criar_datasets_treino_teste_final(
    X_features=X_features_e, 
    y_target=y_target_e
)

# 2. Processar os Dados do MODO B
df_train_B, df_test_B = criar_datasets_treino_teste_final(
    X_features=X_features_b, 
    y_target=y_target_b, 
)

print(f"Modo E - Treino/Teste Shapes: {df_train_E.shape} / {df_test_E.shape}")
print(f"Modo B - Treino/Teste Shapes: {df_train_B.shape} / {df_test_B.shape}")

## Salvando dados de treino e teste em um arquivo csv

In [ ]:
df_train_E.to_csv('../data/02_intermediate/training_e_df.csv', index=False)
df_test_E.to_csv('../data/02_intermediate/teste_e_df.csv', index=False)

df_train_B.to_csv('../data/02_intermediate/training_b_df.csv', index=False)
df_test_B.to_csv('../data/02_intermediate/teste_b_df.csv', index=False)

# Checa salvamento dos dados

In [ ]:
df_e = pd.read_csv('../data/02_intermediate/training_e_df.csv')
df_e_teste = pd.read_csv('../data/02_intermediate/teste_e_df.csv')
df_b = pd.read_csv('../data/02_intermediate/training_b_df.csv')
df_b_teste = pd.read_csv('../data/02_intermediate/teste_b_df.csv')

In [ ]:
df_e

In [ ]:
df_e_teste

In [ ]:
df_b

In [ ]:
df_b_teste